In [ ]:
import pixivpy3 as pixiv

In [ ]:
api = pixiv.AppPixivAPI()

In [ ]:
api.auth(refresh_token='<...>')

In [ ]:
USER_ID = 18218607

In [ ]:
def handle_err(res):
    if 'error' in res:
        print('error:', res.error)
        return True
    else:
        return False

In [ ]:
illusts = []
qs = {'user_id': USER_ID}
while qs is not None:
    res = api.user_bookmarks_illust(**qs)
    if handle_err(res):
        print('qs:', qs)
        break
    if res.illusts is not None:
        illusts.extend(res.illusts)
    qs = api.parse_qs(res.next_url)
print(len(illusts))

In [ ]:
print(illusts[0].title)
print(illusts[-1].title)

In [ ]:
import json

In [ ]:
with open('export/fav_illusts.json', 'w') as f:
    json.dump(illusts, f)

In [ ]:
illust_urls = {}
for illust in illusts:
    if 'meta_pages' in illust and len(illust['meta_pages']) > 0:
        urls = []
        for page in illust['meta_pages']:
            if 'image_urls' not in page:
                print(page)
            urls.append(page['image_urls']['original'])
        illust_urls[illust['id']] = urls
    else:
        illust_urls[illust['id']] = [illust['meta_single_page']['original_image_url']]

In [ ]:
import pathlib

In [ ]:
illust_tasks = list(illust_urls.items())
for i, (id, urls) in enumerate(illust_tasks):
    if len(urls) <= 1:
        try:
            api.download(urls[0], path='export')
        except:
            print('failed at index %d id %s' % (i, id))
            raise
    else:
        pathlib.Path('export/%s' % id).mkdir(exist_ok=True)
        for j, url in enumerate(urls):
            try:
                api.download(url, path='export/%s' % id)
            except:
                print('failed at index %d-%d id %s' % (i, j, id))
                raise

- Remove images with the `limit_` prefix manually, which are not illustrations
- For images with the `_ugoira0` suffix manually, they are ugoiras and need special handling to get the zip pack

In [ ]:
ugoira_ids = [illust['id'] for illust in illusts if illust['type'] == 'ugoira']

In [ ]:
ugoiras = [api.ugoira_metadata(id)['ugoira_metadata'] for id in ugoira_ids]

In [ ]:
ugoira_meta = {ugoira_ids[i]: ugoira for i, ugoira in enumerate(ugoiras)}

In [ ]:
with open('export/fav_ugoira_meta.json', 'w') as f:
    json.dump(ugoira_meta, f)

In [ ]:
ugoira_urls = {ugoira_ids[i]: ugoira['zip_urls']['medium'] for i, ugoira in enumerate(ugoiras)}

In [ ]:
# To get the original (or at least larger) ugoira
ugoira_urls = {id: url.replace('600x600', '1920x1080') for id, url in ugoira_urls.items()}

In [ ]:
for _, url in ugoira_urls.items():
    api.download(url, path='export')